# Create dummy inputs and store in the correct model directory 
For the moment want to create dummy input data, and keep this constant such that it does not change. Use this data to verify that the hw + cpu run has the same output as the expected output.

In [10]:
import numpy as np
import os

# Get expected output 
For each of the inputs get the expected output from the baseline model

In [26]:
from qonnx.core.modelwrapper import ModelWrapper
from onnxruntime_extensions import get_library_path
import onnxruntime as ort
import sys 
from qonnx.transformation.double_to_single_float import DoubleToSingleFloat
sys.path.insert(1,"../src/")
from custom_ort_functions import set_multithreshold_default

mf = "../models/mobilenet_streamline.onnx"
model = ModelWrapper(mf)
model = model.transform(DoubleToSingleFloat())
# for n in model.graph.node:
#     if n.name == "AveragePool_0":
#         print(n.attribute)

set_multithreshold_default(model,mf.replace('.onnx',"_ort.onnx"))



so = ort.SessionOptions()
so.register_custom_ops_library(get_library_path())
sess = ort.InferenceSession(mf.replace('.onnx',"_ort.onnx"), so)
inp_shape = sess.get_inputs()[0].shape
inp_name = sess.get_inputs()[0].name
inp_type = sess.get_inputs()[0].type

dtype = ""
if "float" in inp_type:
    dtype = np.float32
elif "int" in inp_type:
    dtype = np.int
inp_dict = {inp_name: np.random.random(inp_shape).astype(dtype)}

res = sess.run([],inp_dict)
res

[array([[644, 818, 530, 111, 626]], dtype=int64)]